# BT02: Ẩn tin mật trên ảnh gif (ảnh palette-based) bằng phương pháp LSB


Họ tên: Hoàng Minh Thanh

MSSV: 18424062

---
## Cách làm bài và nộp bài
&#9889; Bạn lưu ý là mình sẽ dùng chương trình hỗ trợ chấm bài nên bạn cần phải tuân thủ chính xác qui định mà mình đặt ra, nếu không rõ thì hỏi, chứ không nên tự tiện làm theo ý của cá nhân.

**Cách làm bài**

Bạn sẽ làm trực tiếp trên file notebook này. Đầu tiên, bạn điền họ tên và MSSV vào phần đầu file ở bên trên. Trong file, bạn làm bài ở những chỗ có ghi là:
```python
# YOUR CODE HERE
raise NotImplementedError()
```
hoặc đối với những phần code không bắt buộc thì là:
```python
# YOUR CODE HERE (OPTION)
```
hoặc đối với markdown cell thì là:
```markdown
YOUR ANSWER HERE
```
Tất nhiên, khi làm thì bạn xóa dòng `raise NotImplementedError()` đi.
Đối những phần yêu cầu code thì thường ở ngay phía dưới sẽ có một (hoặc một số) cell chứa các bộ test để giúp bạn biết đã code đúng hay chưa; nếu chạy cell này không có lỗi gì thì có nghĩa là qua được các bộ test. Trong một số trường hợp, các bộ test có thể sẽ không đầy đủ; nghĩa là, nếu không qua được test thì là code sai, nhưng nếu qua được test thì chưa chắc đã đúng hoàn toàn.

Trong khi làm bài, bạn có thể cho in ra màn hình, tạo thêm các cell để test. Nhưng khi nộp bài thì bạn xóa các cell mà bạn tự tạo, xóa hoặc comment các câu lệnh in ra màn hình. Bạn lưu ý <font color=red>không được tự tiện xóa các cell hay sửa code của Thầy</font> (trừ những chỗ được phép sửa như đã nói ở trên).

Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.


*Nên nhớ mục tiêu chính ở đây là <font color=green>học, học một cách chân thật</font>. Bạn có thể thảo luận ý tưởng với bạn khác, nhưng <font color=green>code và bài làm phải là của bạn, dựa trên sự hiểu thật sự của bạn</font>. <font color=red>Nếu vi phạm thì sẽ bị 0 điểm cho toàn bộ môn học.</font>*

**Cách nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart Kernel & Run All Cells`, để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart Kernel & Run All Cells` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, bạn tạo thư mục nộp bài theo cấu trúc sau:
- Thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`)
    - Thư mục `BT02`
        - File `BT02-AnTinMatTrenAnhGif_LSB.ipynb` (không cần nộp các file khác)

Cuối cùng, bạn nén thư mục `MSSV` này lại và nộp ở link trên moodle. <font color=red>Bạn lưu ý tuân thủ chính xác cấu trúc này.</font>

---
## Import

In [41]:
from bitarray import bitarray
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# YOUR CODE HERE (OPTION)
import math

---
## Định nghĩa các hàm hỗ trợ (nếu cần)

In [40]:
# YOUR CODE HERE (OPTION)
def get_icolor_bit(icolor, palette):
    color = palette[icolor]
    
    subtract_distance = np.subtract(palette, color, dtype=np.int16)
    power_distance = np.power(subtract_distance, 2, dtype=np.int32)
    distance_palette = np.array(np.sqrt(np.sum(power_distance, axis=1)), dtype=np.float16) # max value is sqrt(255*255*3) = 441 while max float16 is 65500.0
        
    bit = np.sum(color, axis=0) % 2
    max_float16 = np.finfo(np.float16).max
    if bit == 0:
        argmin_bit0 = icolor
        mask_bit0 = np.sum(palette, axis=1) % 2 == 0 # set all bit 1 to INF
        distance_palette[mask_bit0] = max_float16
        distance_palette[icolor] = max_float16
        
        argmin_bit1 = distance_palette.argmin().astype(np.uint8)
    else:
        argmin_bit1 = icolor
        mask_bit1 = np.sum(palette, axis=1) % 2 == 1
        distance_palette[mask_bit1] = max_float16
        distance_palette[icolor] = max_float16
        
        argmin_bit0 = distance_palette.argmin().astype(np.uint8)        

    return (argmin_bit0, argmin_bit1)

---
## Hàm nhúng (7đ, trong đó: phần code 5đ, phần nhận xét và giải thích kết quả 2đ)

&#9889; Lưu ý:
- Khi tính toán bạn cần để ý kiểu dữ liệu. Ví dụ, nếu lấy 2 giá trị có kiểu dữ liệu `uint8` trừ cho nhau thì kết quả cũng sẽ có kiểu `uint8` và sẽ bị sai.
- Qui ước (để mình và bạn ra cùng kết quả): nếu trong bảng màu có nhiều màu thỏa điều kiện tìm kiếm thì lấy màu có chỉ số nhỏ nhất trong bảng màu. Nếu bạn có dùng hàm `sort` hay `argsort` của numpy thì bạn cần truyền tham số `kind="stable"`; nếu không thì các phần tử có giá trị giống nhau có thể sẽ bị thay đổi thứ tự tương đối với nhau trong quá trình sort.

In [42]:
def embed(msg_file, cover_img_file, stego_img_file):
    '''
    Nhúng tin mật vào ảnh gif (ảnh palette-base) bằng phương pháp của Fridrich 
    (xem file slide "06-AnTinMatTrenAnh2.pdf", trang 24).
    
    Các tham số:
        msg_file (str): Tên file chứa secret message.
        cover_img_file (str): Tên file chứa cover image.
        stego_img_file (str): Tên file chứa stego image (kết quả sau khi nhúng).
    Giá trị trả về:
        bool: True nếu nhúng thành không, False nếu không đủ chỗ để nhúng. 
    '''
    # Đọc cover img file
    cover_img = Image.open(cover_img_file)
    cover_pixels = np.array(cover_img)
    palette = cover_img.getpalette() # Kết quả là list các giá trị Red, Green, Blue của các màu 
                                     # trong bảng màu, 3 giá trị liên tiếp ứng với một màu
    palette = np.array(palette, dtype=np.uint8).reshape(1, -1, 3) # Reshape lại dưới dạng ảnh RGB
    #plt.figure(); plt.yticks([]); plt.imshow(palette, aspect=20) # Uncomment để xem bảng màu
    
    # YOUR CODE HERE
    # Lấy thông tin cover image
    width, height = cover_pixels.shape
    reshape_pixels = cover_pixels.reshape(-1)
    palette = np.squeeze(np.array(palette, copy=True))
    
    # Đọc file msg và chuyển thành chuỗi bit
    with open(msg_file, 'r') as f:
        msg = f.read()
    msg_bits = bitarray()
    msg_bits.fromstring(msg)
    msg_bits.append(True)
    zero_nums = width * height - msg_bits.length()
    if zero_nums <= 0:
        return False
    msg_bits.extend([False] * zero_nums)
    numpy_msg_bits = np.array(msg_bits.tolist(), dtype=np.ushort)
    
    # Tạo bảng lookup table cho các màu
    get_icolor_bits = np.vectorize(get_icolor_bit, excluded=['palette'])
    # [0.255][0.255] ~ [lookup table bit 0][lookup table bit 1]
    lookup_bit_table = np.array(get_icolor_bits(icolor=np.arange(256), palette=palette), dtype=np.uint8)
    
    # Nhúng msg vào cover image
    stego_pixels = lookup_bit_table[numpy_msg_bits, reshape_pixels]
    stego_pixels = stego_pixels.reshape((width, height))
    
    # Ghi stego pixels cùng palette xuống file
    stego_img = Image.fromarray(stego_pixels)
    stego_img.putpalette(cover_img.getpalette())
    stego_img.save(stego_img_file)
    
    return True

In [43]:
# TEST 
result = embed('msg2.txt', 'lena.gif', 'lena_stego.gif')
assert result == False

In [44]:
# TEST             
result = embed('msg.txt', 'lena.gif', 'lena_stego.gif')
assert result == True
stego_img = Image.open('lena_stego.gif')
stego_pixels = np.array(stego_img)
stego_palette = stego_img.getpalette()
correct_stego_img = Image.open('correct_lena_stego.gif')
correct_stego_pixels = np.array(correct_stego_img)
correct_stego_palette = correct_stego_img.getpalette()
assert np.all(stego_pixels == correct_stego_pixels)
assert stego_palette == correct_stego_palette

In [45]:
# TEST             
result = embed('msg.txt', 'baboon.gif', 'baboon_stego.gif')
assert result == True
stego_img = Image.open('lena_stego.gif')
stego_pixels = np.array(stego_img)
stego_palette = stego_img.getpalette()
correct_stego_img = Image.open('correct_lena_stego.gif')
correct_stego_pixels = np.array(correct_stego_img)
correct_stego_palette = correct_stego_img.getpalette()
assert np.all(stego_pixels == correct_stego_pixels)
assert stego_palette == correct_stego_palette

Xem các ảnh và so sánh tính vô hình khi nhúng trên ảnh "lena.gif" với khi nhúng trên ảnh "baboon.gif". Theo bạn, tại sao lại như vậy? (Gợi ý: trong hàm nhúng ở trên, bạn có thể tính và cho in ra giá trị trung bình của khoảng cách từ mỗi màu trong bảng màu tới màu gần nhất mà có (R+G+B)%2 khác với (R+G+B)%2 của mình)

YOUR ANSWER HERE
### Trả lời
Vì trong ảnh `lena.gif` thì ảnh có màu sắc được phân bố một cách liền mạch, nên khi nhúng bit vào sẽ tạo những đốm li ti khác biệt do phải chèn màu khác so với màu gốc trên một dài màu liên tục của ảnh gốc
Còn trong hình `baboon.gif` thì ảnh có màu sắc được phân bố rời rạc thì khi chèn bit nhúng vào thì những đốm li ti cho chèn bit nhúng theo mắt người thì không thấy có sự khác biệt rõ rệt

---
## Hàm rút trích (3đ)

In [46]:
def extract(stego_img_file, extr_msg_file):
    '''
    Hàm rút trích tin mật đã được nhúng vào ảnh bằng phương pháp của Fridrich.
    
    Các tham số:
        stego_img_file (str): Tên file chứa stego image.
        extr_msg_file (str): Tên file chứa secret message được rút trích.
    '''
    # YOUR CODE HERE
    stego_img = Image.open(stego_img_file)
    stego_pixels = np.array(stego_img)
    palette = stego_img.getpalette()
    palette = np.array(palette, dtype=np.uint8).reshape(-1, 3)
    stego_pixels = stego_pixels.reshape(-1)
    colors_pixels = palette[stego_pixels]
    list_bits = np.sum(colors_pixels, axis=1) % 2
    ilast_bit1 = np.argwhere(list_bits==1).max()
    list_abc = list_bits[:ilast_bit1].tolist()
    stego_bits = bitarray(list_abc)
    stego_text = stego_bits.tostring()
    
    with open(extr_msg_file, 'w') as f:
        f.write(stego_text)

In [47]:
# TEST
extract('correct_baboon_stego.gif', 'extr_msg.txt')
with open('extr_msg.txt', 'r') as f:
    extr_msg = f.read()
with open('msg.txt', 'r') as f:
    correct_extr_msg = f.read()
assert extr_msg == correct_extr_msg